# Skenario Robot di Dalam Gudang
Perusahaan e-commerce yang berkembang sedang membangun gudang baru, dan perusahaan ingin semua operasi pengambilan di gudang baru dilakukan oleh robot.

In [1]:
import numpy as np

## Mendefinisikan Environment
Environment terdiri dari states, actions, dan rewards. States dan actions adalah input untuk agen AI, sedangkan actions yang mungkin adalah output agen AI.

#### States
States di environment adalah semua kemungkinan lokasi di dalam gudang. Beberapa dari lokasi ini adalah untuk menyimpan barang (kotak hitam), sedangkan lokasi lainnya adalah lorong yang dapat digunakan robot untuk berjalan di seluruh gudang (kotak putih). Kotak hijau menunjukkan area pengemasan dan pengiriman barang.

Kotak hitam dan hijau adalah terminal states!

![QLearning1](QLearning1.png)

Tujuan agen AI adalah mempelajari jalur terpendek antara area pengemasan barang dan semua lokasi lain di gudang tempat robot diizinkan untuk bepergian.

Seperti yang ditunjukkan pada gambar di atas, ada 25 kemungkinan states (lokasi) di dalam gudang. States ini diatur dalam kisi yang berisi 5 baris dan 5 kolom. Setiap lokasi dapat diidentifikasi dengan indeks baris dan kolomnya.

In [2]:
baris_environment=5
kolom_environment=5

nilai_nilai_q=np.zeros((baris_environment,kolom_environment, 4))

#### Actions
Actions yang tersedia untuk agen AI adalah menggerakkan robot ke salah satu dari empat arah:
* Atas
* Kanan
* Bawah
* Kiri

Agen AI harus belajar untuk menghindari mengemudi ke lokasi penyimpanan barang (misalnya rak)!

In [3]:
actions=['atas','kanan','bawah','kiri']

#### Rewards
Komponen terakhir dari environment yang perlu kita definisikan adalah **rewards**.

Untuk membantu agen AI belajar, setiap states (lokasi) di gudang diberi nilai hadiah.

Agen boleh mulai dari kotak putih mana saja, tetapi tujuannya selalu sama: ***untuk memaksimalkan total hadiah***!

Imbalan negatif (yaitu, **hukuman**) digunakan untuk semua keadaan kecuali tujuan.
Hal ini mendorong AI untuk mengidentifikasi *jalur terpendek* ke tujuan dengan *meminimalkan hukumannya*!

![QLearning2](QLearning2.png)

Untuk memaksimalkan hadiah kumulatifnya (dengan meminimalkan hukuman kumulatifnya), agen AI perlu menemukan jalur terpendek antara area pengemasan barang (kotak hijau) dan semua lokasi lain di gudang tempat robot diizinkan untuk bepergian (kotak putih ). Agen juga perlu belajar untuk menghindari menabrak salah satu lokasi penyimpanan item (kotak hitam)!

In [4]:
rewards=np.full((baris_environment,kolom_environment), -100.)
rewards[0,2]=100.

lorong={}
lorong[1]=[i for i in range(5)]
lorong[2]=[0,3]
lorong[3]=[i for i in range(4)]
lorong[4]=[1,3,4]

for baris_index in range(1,5):
    for kolom_index in lorong[baris_index]:
        rewards[baris_index, kolom_index]= -1.


for baris in rewards:
    print(baris)

[-100. -100.  100. -100. -100.]
[-1. -1. -1. -1. -1.]
[  -1. -100. -100.   -1. -100.]
[  -1.   -1.   -1.   -1. -100.]
[-100.   -1. -100.   -1.   -1.]


## Fungsi-fungsi Pembantu

In [5]:
# fungsi untuk mengecek apakah kolom saat ini adalah terminal state
def is_terminal_state(baris_index_saat_ini, kolom_index_saat_ini):
    if rewards[baris_index_saat_ini, kolom_index_saat_ini] == -1.:
        return False
    else:
        return True

# fungsi untuk memilih lokasi awal non-terminal secara acak
def get_starting_location():
    baris_index_saat_ini = np.random.randint(baris_environment)
    kolom_index_saat_ini = np.random.randint(kolom_environment)
    while is_terminal_state(baris_index_saat_ini, kolom_index_saat_ini):
        baris_index_saat_ini = np.random.randint(baris_environment)
        kolom_index_saat_ini = np.random.randint(kolom_environment)
    return baris_index_saat_ini, kolom_index_saat_ini

# fungsi untuk membuat sebuah algoritma epsilon greedy yang akan memilih tindakan mana yang akan diambil selanjutnya (ke mana agent harus pindah selanjutnya)
def get_next_action(baris_index_saat_ini, kolom_index_saat_ini, epsilon):
    if np.random.random() < epsilon:
        return np.argmax(nilai_nilai_q[baris_index_saat_ini, kolom_index_saat_ini])
    else:
        return np.random.randint(4)

# fungsi untuk mendapatkan lokasi berikutnya berdasarkan tindakan yang dipilih
def get_next_location(baris_index_saat_ini, kolom_index_saat_ini, action_index):
    baris_index_baru = baris_index_saat_ini
    kolom_index_baru = kolom_index_saat_ini
    if actions[action_index] == 'atas' and baris_index_saat_ini > 0:
        baris_index_baru -= 1
    elif actions[action_index] == 'kanan' and kolom_index_saat_ini < kolom_environment - 1:
        kolom_index_baru += 1
    elif actions[action_index] == 'bawah' and baris_index_saat_ini < baris_environment - 1:
        baris_index_baru += 1
    elif actions[action_index] == 'kiri' and kolom_index_saat_ini > 0:
        kolom_index_baru -= 1
    return baris_index_baru, kolom_index_baru

# fungsi untuk mendapatkan jalur terpendek antara lokasi mana pun di dalam gudang
def get_shortest_path(start_baris_index, start_kolom_index):
    if is_terminal_state(start_baris_index, start_kolom_index):
        return []
    else:
        baris_index_saat_ini, kolom_index_saat_ini = start_baris_index, start_kolom_index
        jalur_terpendek = []
        jalur_terpendek.append([baris_index_saat_ini, kolom_index_saat_ini])
        while not is_terminal_state(baris_index_saat_ini, kolom_index_saat_ini):
            action_index = get_next_action(baris_index_saat_ini, kolom_index_saat_ini, 1.)
            baris_index_saat_ini, kolom_index_saat_ini = get_next_location(baris_index_saat_ini, kolom_index_saat_ini, action_index)
            jalur_terpendek.append([baris_index_saat_ini, kolom_index_saat_ini])
        return jalur_terpendek

## Melatih AI Agent menggunakan Q-Learning

In [6]:
# training parameters
epsilon = 0.9 # persentase waktu saat agan AI harus mengambil tindakan terbaik (bukan tindakan acak)
discount_factor = 0.9 # untuk rewards di masa depan
learning_rate = 0.9 # tingkat dimana agen AI harus belajar

for episode in range(1000):
    baris_index, kolom_index = get_starting_location()
    while not is_terminal_state(baris_index, kolom_index):
        action_index = get_next_action(baris_index, kolom_index, epsilon)
        
        baris_index_lama, kolom_index_lama = baris_index, kolom_index
        baris_index, kolom_index = get_next_location(baris_index, kolom_index, action_index)
        
        reward = rewards[baris_index, kolom_index]
        nilai_q_lama = nilai_nilai_q[baris_index_lama, kolom_index_lama, action_index]
        perbedaan_temporal = reward + (discount_factor + np.max(nilai_nilai_q[baris_index, kolom_index])) - nilai_q_lama
        
        nilai_q_baru = nilai_q_lama + (learning_rate * perbedaan_temporal)
        nilai_nilai_q[baris_index_lama, kolom_index_lama, action_index] = nilai_q_baru
        
print('Training Selesai')

Training Selesai


## Mendapatkan Jalur Terpendek

In [7]:
print(get_shortest_path(4,4))

[[4, 4], [4, 3], [3, 3], [2, 3], [1, 3], [1, 2], [0, 2]]


In [8]:
print(get_shortest_path(4,1))

[[4, 1], [3, 1], [3, 0], [2, 0], [1, 0], [1, 1], [1, 2], [0, 2]]


## Skenario yang Berlawanan

In [9]:
path = get_shortest_path(3,0)
path.reverse()
print(path)

[[0, 2], [1, 2], [1, 1], [1, 0], [2, 0], [3, 0]]
